In [1]:
import serial 
from time import sleep
ser = serial.Serial(
    port="COM2",\
    baudrate=9600,\
    parity=serial.PARITY_NONE,\
    stopbits=serial.STOPBITS_ONE,\
    bytesize=serial.EIGHTBITS,\
    timeout=0)

print("connected to: " + ser.portstr)
oldline = ""
try:
    while True:
        line = ser.readline()
        
        if oldline != line and len(line)>1:
            print(''.join(" " + '\\x{:02x}'.format(letter) for letter in line))
            oldline = line
            if data_from_serial(line):
                print("success")
            if data == bytearray((0x00, 0x01, 0x01, 0x00, 0x01)):
                ser.write(bytearray((0xa5, 0x5a, 0x07, 0x82, 0x00, 0x01, 0x15, 0x23, 0x42, 0x97)))
                print("make online")
            if data == bytearray((0x00, 0x03, 0x01, 0x00, 0x03)):
                print("Goto ESD 1")
                ser.flush()
                sleep(1)
                ser.write(bytearray((0xa5, 0x5a, 0x07, 0x82, 0x00, 0x03, 0x00, 0x03, 0xec, 0x1f)))
                print("Goto ESD 2")
            #ser.write(b'82000188886bb8')
            
            print(ser.in_waiting)
            print(ser.out_waiting)
except KeyboardInterrupt:
    print('interrupted!')    
    
ser.close()


SerialException: could not open port 'COM2': FileNotFoundError(2, 'The system cannot find the file specified.', None, 2)

In [3]:
ser.close()

In [7]:
'''
IN THIS CELL THERE ARE ALL THE HELPER FUNCTIONS AND IMPORTS
'''
import crcmod
import serial 
from time import sleep
from time import perf_counter

class colors:
    black = "\033[90m"
    red = "\033[91m" 
    green = "\033[92m" 
    yellow = "\033[93m" 
    blue = "\033[94m" 
    magenta = "\033[95m" 
    cyan = "\033[96m" 
    white = "\033[97m"
    reset = "\033[0m"
    info = "\033[36m"
    warn = "\033[33m"
    reading = "\033[32m" 
    error = "\033[91m" 

def list_comports():
    import serial.tools.list_ports
    ports = serial.tools.list_ports.comports()

    if ports:
        for port, desc, hwid in sorted(ports):
            print("{}: {} [{}]".format(port, desc, hwid))
    else:
        print("No COM ports available")
        
def calculate_crc(data):
    crc16 = crcmod.mkCrcFun(0x18005, rev=True, initCrc=0xFFFF, xorOut=0x0000)
    #print(crc16(bytearray((0x83,0x00,0x02,0x01,0x00,0x02))).to_bytes(2,'little')) #'little' flips the bytes to be in the correct "direction"
    return crc16(data).to_bytes(2,'little') #'little' flips the bytes to be in the correct "direction"

def print_bytearray(arr, sr=None):
    #print(''.join(" " + '\\x{:02x}'.format(letter) for letter in arr))
    if sr:
        if sr=="s":
            print(''.join(" " + '{:02x}'.format(letter) for letter in arr) + " -> SENT")
        elif sr=="r":
            print(''.join(" " + '{:02x}'.format(letter) for letter in arr) + " -> RECIEVED")
        elif sr=="e":
            print(colors.red + ''.join(" " + '{:02x}'.format(letter) for letter in arr) + " -> ERROR IN THIS LINE" + colors.reset)
        else:
            print(''.join(" " + '{:02x}'.format(letter) for letter in arr))
    else: 
        print(''.join(" " + '{:02x}'.format(letter) for letter in arr))


In [31]:
def select_function(function):
    global curr_mode
    
    if function == "ESD":
        
        #Do something and wait for confirmation from equipment
        data = ser_direction + bytearray([curr_mode]) + bytearray((0x03, 0x01, 0x00, 0x03))
        line_to_send = ser_head + bytearray([0x08]) + data + calculate_crc(data)
        print_bytearray(line_to_send, "s")
        ser.write(line_to_send)
        
        old_curr_mode = curr_mode
        curr_mode = 3
        '''
        test_data = bytearray([0x82]) + bytearray([old_curr_mode]) + bytearray((0x03, 0x00)) + bytearray([curr_mode])
        test_indata = ser_head + bytearray([0x07]) + test_data + calculate_crc(test_data)
        decode("waveform", line_to_send, test_indata)
        ''' 
        if read_serial(ser):
            if decode("waveform", line_to_send, ser_read):
                return True
            else:
                print("Error decoding the recieved message")
                return False
        else: 
            print(colors.yellow + "Error reading from the serial" + colors.reset)
            return False
        
    elif function == "EFT":
        #Do nothing for now since only ESD is available in the beginning
        pass
    elif function == "SURGE":
        #Do nothing for now since only ESD is available in the beginning
        pass
    else:
        print("Unknown function inputted. Nothing done")
        

def read_serial(ser):
    global ser_read
    if not ser:
        return True
    tic = perf_counter()
    while True:
        if(ser.in_waiting > 0):
            sleep(0.05)
            ser_read = ser.read(ser.in_waiting)
            print_bytearray(ser_read, "r")
            return True
        if(perf_counter() > tic + 5):
            print(colors.yellow + "Serial read timeout" + colors.reset)
            return False
        sleep(0.05)
    return False

def read_serial_esd(ser):
    print(colors.info + "Start reading while testing" + colors.reset)
    global temp
    global humidity
    tic = perf_counter()
    while True:
        if(ser.in_waiting >= 10):
            read_line = ser.read(10)
            print_bytearray(read_line, "r")
            tic = perf_counter()
            #do something with this countdown and number of trials
            if(read_line[0:6] == bytearray((0xA5, 0x5A, 0x07, 0x82, 0x03, 0x08))):
                temp = read_line[6]
                humidity = read_line[7]
                print(colors.info + "Stop reading while started" + colors.reset)
                print(colors.reading + "current temperature is " + str(temp) + colors.reset)
                print(colors.reading + "current humidity is " + str(humidity) + colors.reset)
                return True
            
        if(perf_counter() > tic + 30):
            print(colors.yellow + "Serial read timeout" + colors.reset)
            return False
        sleep(0.005)
        

#on : 01
#off : 02
def onoffline(onoff):
    data = ser_direction +  bytearray([curr_mode]) + bytearray((0x01, 0x01, 0x00)) + onoff
    line_to_send = ser_head + bytearray([0x08]) + data + calculate_crc(data)
    print_bytearray(line_to_send, "s")
    ser.write(line_to_send)
    '''
    test_data = bytearray([0x82]) + bytearray([curr_mode]) + onoff + bytearray((0x15, 0x15))
    test_indata = ser_head + bytearray([0x07]) + test_data + calculate_crc(test_data)
    
    decode("onoff", line_to_send, test_indata)
    '''
    if read_serial(ser):
        if decode("onoff", line_to_send, ser_read):
            return True
        else:
            print("Error decoding the recieved message")
            return False
    else: 
        print(colors.yellow + "Error reading from the serial" + colors.reset)
        return False
    
def exit():
    data = ser_direction +  bytearray([curr_mode]) + bytearray((0x02, 0x01, 0x00, 0x02))
    line_to_send = ser_head + bytearray([0x08]) + data + calculate_crc(data)
    print_bytearray(line_to_send, "s")
    ser.write(line_to_send)
    '''
    test_data = bytearray([0x82]) + bytearray([curr_mode]) + bytearray((0x02, 0x00, 0x02))
    test_indata = ser_head + bytearray([0x07]) + test_data + calculate_crc(test_data)
    
    decode("exit", line_to_send, test_indata)
    '''
    if read_serial(ser):
        if decode("exit", line_to_send, ser_read):
            return True
        else:
            print("Error decoding the recieved message")
            return False
    else: 
        print(colors.yellow + "Error reading from the serial" + colors.reset)
        return False
    
def start():
    data = ser_direction +  bytearray([curr_mode]) + bytearray((0x05, 0x01, 0x00, 0x05))
    line_to_send = ser_head + bytearray([0x08]) + data + calculate_crc(data)
    print_bytearray(line_to_send, "s")
    ser.write(line_to_send)
    '''
    test_data = bytearray([0x82]) + bytearray([curr_mode]) + bytearray((0x05, 0x11, 0x22))
    test_indata = ser_head + bytearray([0x07]) + test_data + calculate_crc(test_data)
    
    decode("start", line_to_send, test_indata)
    '''
    if read_serial(ser):
        if decode("start", line_to_send, ser_read):
            return True
        else:
            print("Error decoding the recieved message")
            return False
    else: 
        print(colors.yellow + "Error reading from the serial" + colors.reset)
        return False
    
def stop():
    data = ser_direction +  bytearray([curr_mode]) + bytearray((0x06, 0x01, 0x00, 0x06))
    line_to_send = ser_head + bytearray([0x08]) + data + calculate_crc(data)
    print_bytearray(line_to_send, "s")
    ser.write(line_to_send)
    '''
    test_data = bytearray([0x82]) + bytearray([curr_mode]) + bytearray((0x06, 0x22, 0x11))
    test_indata = ser_head + bytearray([0x07]) + test_data + calculate_crc(test_data)
    
    decode("stop", line_to_send, test_indata)
    '''
    if read_serial(ser):
        if decode("stop", line_to_send, ser_read):
            return True
        else:
            print("Error decoding the recieved message")
            return False
    else: 
        print(colors.yellow + "Error reading from the serial" + colors.reset)
        return False
'''
Write data to the machine when in the ESD test
'''
def write_esd(pol, volt, method, dis_type, interval, number):
    sleep(2)
    if(curr_mode != 3):
        print(colors.yellow + "Can not write ESD in any mode besides the ESD mode" + colors.reset) 
        return False
    
    #Start to see if any of the recieved parameters are out of bounds
    if(type(pol) != int or pol > 4 or pol < 1):
        print(colors.yellow + "polarity must be a integer on the range 1-4" + colors.reset)
        return False
    if(volt > 20000 or volt < 200):
        print(colors.yellow + "voltage must be on the range of 200 - 20000 (volts)" + colors.reset)
        return False
    if(type(method) != int or method > 2 or method < 1):
        print(colors.yellow + "method must be a integer on the range 1-2" + colors.reset)
        return False
    if(type(dis_type) != int or dis_type > 4 or dis_type < 1):
        print(colors.yellow + "discharge type must be a integer on the range 1-4" + colors.reset)
        return False
    if(interval > 9.99 or interval < 1.0):
        print(colors.yellow + "interval must be on the range of 1.00 - 9.99s" + colors.reset)
        return False
    if(type(number) != int or number > 9999 or number < 1):
        print(colors.yellow + "number must be a integer on the range 1-9999" + colors.reset)
        return False
    
    
    #change all floats to integers and have on the correct range
    volt = int(volt / 100)
    #if single discharge, then no interval possible
    if(dis_type == 1 and interval > 0):
        interval = 0
        print(colors.info + "Interval changed to 0" + colors.reset)
    else:
         interval = int(interval * 100)
    
    
   
    
    #Construct the data string
    data = ser_direction + bytearray([curr_mode]) + bytearray((0x01, 0x08, 0x00, 0x05, 0x00)) \
        + bytearray([pol]) + volt.to_bytes(2,byteorder='big') + bytearray((0x00, 0x00, 0x00)) \
        + bytearray([method]) + bytearray([0x00]) + bytearray([dis_type]) \
        + interval.to_bytes(2,byteorder='big') + number.to_bytes(2,byteorder='big') 
    
    line_to_send = ser_head + bytearray([len(data)+2]) + data + calculate_crc(data)
    print_bytearray(line_to_send, "s")
    ser.write(line_to_send)
    '''
    #Test receive
    test_data = bytearray([0x82]) + bytearray([curr_mode]) + bytearray((0x07, 0x12, 0x16))
    test_indata = ser_head + bytearray([0x07]) + test_data + calculate_crc(test_data)
    
    decode("write", line_to_send, test_indata)
    '''
    if read_serial(ser):
        if decode("write", line_to_send, ser_read):
            return True
        else:
            print("Error decoding the recieved message")
            return False
    else: 
        print(colors.yellow + "Error reading from the serial" + colors.reset)
        return False
    
    

'''
Types to decode:
onoff
exit
waveform
write
eut
start (not the extra commands)
stop

'''
def decode(command_type, out_data_true, in_data_true):
    global temp
    global humidity
    in_data = in_data_true
    out_data = out_data_true
    #Start by stripping of the header (A5 5A) 
    #(if the header is wrong, it lets the user know but keeps on going regardless)
    if(in_data[0:2] != ser_head):
        print("The header is not the standard one, the whole serial incomming string is as follows:")
        print_bytearray(in_data_true, "e")
    in_data = in_data[2:]
    
    out_data = out_data[2:]
    
    
    #look at the length and see if that matches up
    if(in_data[0] != len(in_data[1:])): 
        print("The length of the incoming data is not equal to what is stated in the data:")
        print_bytearray(in_data_true, "e")
         
    #Look at if the outgoing is with 83 and incoming 82
    if(in_data[1].to_bytes(1,byteorder='big') != bytearray([0x82])):
        print("The incoming data is not with the correct COMMAND of 0x82:")
        print_bytearray(in_data_true, "e")
    
    if(out_data[1].to_bytes(1,byteorder='big') != bytearray([0x83])):
        print("The outgoing data is not with the correct COMMAND of 0x83:")
        print_bytearray(out_data_true, "e")
    
    #Check if the CRC of the incoming data is correct
    if(calculate_crc(in_data[1:in_data[0]-2+1]) != in_data[-2:]):
        print("CRC calibration wrong on the incoming data, is:")
        print_bytearray(in_data[-2:])
        print("It should be:")
        print_bytearray(calculate_crc(in_data[1:in_data[0]-2+1]))
    
    
    
    if(command_type == "onoff"):
        #if(in_data[2:4] == out_data[5:7]):
        if(in_data[2] == out_data[2] and in_data[3] == out_data[6]):
            temp = in_data[4]
            humidity = in_data[5]
            print(colors.info + "on off good" + colors.reset)
            print(colors.reading + "current temperature is " + str(temp) + colors.reset)
            print(colors.reading + "current humidity is " + str(humidity) + colors.reset)
            return True
        else:
            print(colors.warn + "on off not good" + colors.reset)
            print_bytearray(in_data_true, "e")
            return False
        
    elif(command_type == "exit"):
        if(in_data[2:6] == out_data[2:4]+out_data[5:7]):
            print(colors.info + "exit good")
            return True
        else:
            print(colors.warn + "exit not good" + colors.reset)
            print_bytearray(in_data_true, "e")
            return False
        
    elif(command_type == "waveform"):
        if(in_data[2:6] == out_data[2:4]+out_data[5:7]):
            print(colors.info + "waveform good" + colors.reset)
            return True
        else:
            print(colors.warn + "waveform not good" + colors.reset)
            print_bytearray(in_data_true, "e")
            return False
    
    elif(command_type == "write"):
        if(in_data[2] == out_data[2] and in_data[3] == 7):
            temp = in_data[4]
            humidity = in_data[5]
            print(colors.info + "write good" + colors.reset)
            print(colors.reading + "current temperature is " + str(temp) + colors.reset)
            print(colors.reading + "current humidity is " + str(humidity) + colors.reset)
            return True
        else:
            print(colors.warn + "write not good" + colors.reset)
            print_bytearray(in_data_true, "e")
            return False
    
    #Not for ESD test so will be implemented later
    elif(command_type == "eut"):
        pass 
    
    elif(command_type == "start"):
        if(in_data[2:4] == out_data[2:4]):
            temp = in_data[4]
            humidity = in_data[5]
            print(colors.info + "start good" + colors.reset)
            print(colors.reading + "current temperature is " + str(temp) + colors.reset)
            print(colors.reading + "current humidity is " + str(humidity) + colors.reset)
            return True
        else:
            print(colors.warn + "start not good" + colors.reset)
            print_bytearray(in_data_true, "e")
            return False
    
    elif(command_type == "stop"):
        if(in_data[2:4] == out_data[2:4]):
            temp = in_data[4]
            humidity = in_data[5]
            print(colors.info + "stop good" + colors.reset)
            print(colors.reading + "current temperature is " + str(temp) + colors.reset)
            print(colors.reading + "current humidity is " + str(humidity) + colors.reset)
            return True
        else:
            print(colors.warn + "stop not good" + colors.reset)
            print_bytearray(in_data_true, "e")
            return False
         
    else:
        print("Command used is not found in the list. Use some other command when calling this function")
    return False

global ser
try:
    list_comports()
    comnum = int(input("Choose which serial port to connect to (0 means none): "))
    ser = serial.Serial(
        port="COM" + str(comnum),\
        baudrate=9600,\
        parity=serial.PARITY_NONE,\
        stopbits=serial.STOPBITS_ONE,\
        bytesize=serial.EIGHTBITS,\
        timeout=0)

    print("connected to: " + ser.portstr)    

except:
    print("No serial initialized")
    
    
print(" 1  2  3  4  5  6  7  8  9  A  B  C  D  E  F  10 11 12 13 14 15 16 17 18 19")
print("---------------------------------------------------------------------------")
#curr_mode = 0 -> none
#curr_mode = 1 -> Surge
#curr_mode = 2 -> EFT
#curr_mode = 3 -> ESD

global curr_mode
curr_mode = 0

global ser_head
ser_head = bytearray((0xa5, 0x5a)) #Bytearrays can be concatinated with "+" (like strings)

global ser_direction
ser_direction = bytearray([0x83]) #82 is the equipment sending, 83 is the computer sending

global ser_read
ser_read = bytearray([0x0])

'''
#make online
onoffline(bytearray([0x01]))

#select ESD
select_function("ESD")

#Write to ESD test
write_esd(1, 1000, 2, 1, 1, 10)

#Start the test
start()

#Read the serial while the test is ongoing
read_serial_esd(ser)

#Stop the test
stop()

#make offline
onoffline(bytearray([0x02]))

#Quit the computer mode
exit()
'''
if not onoffline(bytearray([0x01])):
    print(colors.error + "Error at Online" + colors.reset)
elif not select_function("ESD"):
    print(colors.error + "Error at ESD" + colors.reset)
elif not write_esd(1, 1000, 2, 1, 1, 2):
    print(colors.error + "Error at write" + colors.reset)
elif not start():
    print(colors.error + "Error at start" + colors.reset)
#Since it is either stop or finish test:
elif not read_serial_esd(ser):
    print(colors.error + "Error at read serial esd" + colors.reset)
    if not stop():
        print(colors.error + "Error at stop" + colors.reset)
    elif not onoffline(bytearray([0x02])):
        print(colors.error + "Error at offline" + colors.reset)
    elif not exit():
        print(colors.error + "Error at exit" + colors.reset)
    
elif not onoffline(bytearray([0x02])):
    print(colors.error + "Error at offline" + colors.reset)
elif not exit():
    print(colors.error + "Error at exit" + colors.reset)

try:
    ser.close()
except:
    pass



COM5: USB-SERIAL CH341A (COM5) [USB VID:PID=1A86:5523 SER= LOCATION=1-5.3]


Choose which serial port to connect to (0 means none):  5


connected to: COM5
 1  2  3  4  5  6  7  8  9  A  B  C  D  E  F  10 11 12 13 14 15 16 17 18 19
---------------------------------------------------------------------------
 a5 5a 08 83 00 01 01 00 01 8e 14 -> SENT
 a5 5a 07 82 00 01 17 25 c3 f5 -> RECIEVED
on off good
current temperature is 23
current humidity is 37
 a5 5a 08 83 00 03 01 00 03 0e 6d -> SENT
 a5 5a 07 82 00 03 00 03 ec 1f -> RECIEVED
waveform good
Interval changed to 0
 a5 5a 16 83 03 01 08 00 05 00 01 00 0a 00 00 00 02 00 01 00 00 00 02 94 17 -> SENT
 a5 5a 07 82 03 07 17 25 23 b0 -> RECIEVED
write good
current temperature is 23
current humidity is 37
 a5 5a 08 83 03 05 01 00 05 ca e7 -> SENT
 a5 5a 07 82 03 05 17 25 82 70 -> RECIEVED
start good
current temperature is 23
current humidity is 37
Start reading while testing
 a5 5a 07 82 01 00 00 0a dd e5 -> RECIEVED
 a5 08 00 00 00 64 00 00 65 df -> RECIEVED
 a5 08 00 00 00 64 00 01 a4 1f -> RECIEVED
 a5 08 00 00 00 64 00 02 e4 1e -> RECIEVED
 a5 5a 07 82 03 08 17 25 13 b3

In [170]:
print_bytearray(ser_head + curr_mode.to_bytes(5,byteorder='big'), "")

 a5 5a 00 00 00 00 03


In [30]:
ser.close()

In [23]:
read_line = bytearray((0xa5, 0x5a, 0x07, 0x82, 0x03, 0x08, 0x17, 0x25, 0x13, 0xb3))
if(read_line[0:6] == bytearray((0xA5, 0x5A, 0x07, 0x82, 0x03, 0x08))):
    print("hello")

hello


In [4]:
def data_from_serial(line):
    global data
    success = False
    if len(line) < 1: 
        return
    
    GEM = line[0:2] #get first two bytes
    '''
    print("GEM: ")
    print(GEM[0] == 0xa5)
    print(GEM[1] == 0x5a)
    '''
    
    
    data_length = line[2]#Get only bit number 3 (the length) (becomes int)
    
    '''
    print("length: " + str(data_length))
    print(type(data_length))
    '''
    
    command = line[3]
    
    data = line[4:3+data_length-2]
    
    CRC_L = line[-2:-1]
    CRC_H = line[-1]
    
    '''
    print("command: ")
    print(command)
    
    print("data: ")
    print(data)
    
    print("CRC_L: ")
    print(CRC_L)
    
    print("CRC_H: ")
    print(CRC_H)
    '''
    success = True
    return success
    

In [1]:
import crcmod
crc16 = crcmod.mkCrcFun(0x18005, rev=True, initCrc=0xFFFF, xorOut=0x0000)
print(crc16(bytearray((0x83,0x00,0x02,0x01,0x00,0x02))).to_bytes(2,'little')) #'little' flips the bytes to be in the correct "direction"

#\x83 \x00 \x01 \x01 \x00 \x01 
#\x 8e \x 14

#\x83 \x00 \x02 \x01 \x00 \x02
#\xce \x51

#82 00 01 88 88
print(crc16(bytearray((0x82,0x00,0x03,0x00,0x03))).to_bytes(2,'little')) #'little' flips the bytes to be in the correct "direction"
#\x6b \xb8

b'\xceQ'
b'\xec\x1f'


In [66]:
try:
    e = int(input("Choose which serial port to connect to (0 means none): "))
except:
    print(colors.red + "Input not supported, no serial port chosen" + colors.reset)
    e = 0

print(colors.info + "error" + colors.reset)
print("error")

Choose which serial port to connect to (0 means none):  6


6
<class 'int'>
error
error


In [19]:
for i in range(120):
    print(str(i) + "\033["+str(i)+"m TEXT \033[0m")

0 TEXT 
1 TEXT 
2 TEXT 
3 TEXT 
4 TEXT 
5 TEXT 
6 TEXT 
7 TEXT 
8 TEXT 
9 TEXT 
10 TEXT 
11 TEXT 
12 TEXT 
13 TEXT 
14 TEXT 
15 TEXT 
16 TEXT 
17 TEXT 
18 TEXT 
19 TEXT 
20 TEXT 
21 TEXT 
22 TEXT 
23 TEXT 
24 TEXT 
25 TEXT 
26 TEXT 
27 TEXT 
28 TEXT 
29 TEXT 
30 TEXT 
31 TEXT 
32 TEXT 
33 TEXT 
34 TEXT 
35 TEXT 
36 TEXT 
37 TEXT 
38 TEXT 
39 TEXT 
40 TEXT 
41 TEXT 
42 TEXT 
43 TEXT 
44 TEXT 
45 TEXT 
46 TEXT 
47 TEXT 
48 TEXT 
49 TEXT 
50 TEXT 
51 TEXT 
52 TEXT 
53 TEXT 
54 TEXT 
55 TEXT 
56 TEXT 
57 TEXT 
58 TEXT 
59 TEXT 
60 TEXT 
61 TEXT 
62 TEXT 
63 TEXT 
64 TEXT 
65 TEXT 
66 TEXT 
67 TEXT 
68 TEXT 
69 TEXT 
70 TEXT 
71 TEXT 
72 TEXT 
73 TEXT 
74 TEXT 
75 TEXT 
76 TEXT 
77 TEXT 
78 TEXT 
79 TEXT 
80 TEXT 
81 TEXT 
82 TEXT 
83 TEXT 
84 TEXT 
85 TEXT 
86 TEXT 
87 TEXT 
88 TEXT 
89 TEXT 
90 TEXT 
91 TEXT 
92 TEXT 
93 TEXT 
94 TEXT 
95 TEXT 
96 TEXT 
97 TEXT 
98 TEXT 
99 TEXT 
100 TEXT 
101 TEXT 
102 TEXT 
103 TEXT 
104 TEXT 
105 TEXT 
106 TEXT 
107 TEXT 
108 TEXT 
109 TEXT 
110 TEXT 


In [18]:
import serial 
import crcmod
ser = serial.Serial(
    port="COM5",\
    baudrate=9600,\
    parity=serial.PARITY_NONE,\
    stopbits=serial.STOPBITS_ONE,\
    bytesize=serial.EIGHTBITS,\
    timeout=0)

print("connected to: " + ser.portstr)

connected to: COM5


In [19]:
data = bytearray((0x83, 0x03, 0x06, 0x01, 0x00, 0x06))
line_to_send = bytearray((0xA5, 0x5A, 0x07)) + data + calculate_crc(data)
print_bytearray(line_to_send, "s")
ser.write(line_to_send)

 a5 5a 07 83 03 06 01 00 06 8a a2 -> SENT


11

In [20]:
ser.close()